<a href="https://colab.research.google.com/github/fabiomatricardi/17megabyteCHATBOT/blob/main/17megabyteChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🦙🧑‍💻 CrossEncoder ChatBot == 17Mb model ⚖️🔬

In [1]:
%%capture
!pip install sentence_transformers
!pip install rich

In [4]:
from tqdm.rich import trange, tqdm
from rich import console
from rich.panel import Panel
from rich.markdown import Markdown
from rich.text import Text
import warnings
warnings.filterwarnings(action='ignore')
import datetime
from rich.console import Console
console = Console(width=110)
from transformers import pipeline
import os
## Logger file
tstamp = datetime.datetime.now()
tstamp = str(tstamp).replace(' ','_')
logfile = f'{tstamp}_log.txt'
def writehistory(text):
    with open(logfile, 'a', encoding='utf-8') as f:
        f.write(text)
        f.write('\n')
    f.close()
listato = [
    {'q': 'Where is Windscribe located?', 'a' : 'The actual Windscribe headquarters is based in Toronto, Ontario, Canada. However, we are a diverse company with employees working on three continents, in over 7 countries.'},
    {'q': 'What is WireGuard?' , 'a': 'WireGuard is a connection protocol used in the Windscribe desktop and mobile applications. It is typically faster than OpenVPN (called UDP and TCP in the apps) and more flexible than IKEv2, making it a great option for securing your online activity.' },
    {'q': 'What kind of encryption does Windscribe use?', 'a' : "Windscribe's encryption varies based on the protocol selected, as well as the format of our app you are using: OpenVPN: Our OpenVPN implementation uses the AES-256-GCM cipher with SHA512 auth and a 4096-bit RSA key. Perfect forward secrecy is also supported. IKEv2: Our in-app IKEv2 implementation utilizes AES-256-GCM for encryption, SHA-256 for integrity checks. Desktop and Android apps use ECP384 for Diffie-Hellman key negotiation (DH group 20), and iOS uses ECP521 for Diffie-Hellman key negotiation (DH group 21). WireGuard: WireGuard is an opinionated protocol that uses ChaCha20 for symmetric encryption, authenticated with Poly1305; Curve25519 for ECDH; BLAKE2s for hashing and keyed hashing; SipHash24 for hashtable keys; and HKDF for key derivation."},
    {'q': 'What is OpenVPN?', 'a' : 'Our OpenVPN implementation uses the AES-256-GCM cipher with SHA512 auth and a 4096-bit RSA key. Perfect forward secrecy is also supported.'},
    {'q': 'What is IKEv2?', 'a' : 'Our in-app IKEv2 implementation utilizes AES-256-GCM for encryption, SHA-256 for integrity checks. Desktop and Android apps use ECP384 for Diffie-Hellman key negotiation (DH group 20), and iOS uses ECP521 for Diffie-Hellman key negotiation (DH group 21).'},
    {'q': 'What is WireGuard?', 'a' : 'WireGuard is an opinionated protocol that uses ChaCha20 for symmetric encryption, authenticated with Poly1305; Curve25519 for ECDH; BLAKE2s for hashing and keyed hashing; SipHash24 for hashtable keys; and HKDF for key derivation.'},
    {'q': 'Can I use Windscribe without an internet connection?', 'a' : 'In order to use Windscribe, you must have an existing internet connection, such as mobile data, home internet or public WiFi. Windscribe does not provide an internet connection, we simply encrypt and reroute your existing internet connection through our secure VPN servers. This means that the bandwidth you use with the VPN is also being used with your Internet Service Provider (ISP).'},

]
questions = []
answers = []
for i in listato:
  questions.append(i['q'])
  answers.append(i['a'])

In [5]:
import time,os,sys

def typingPrint(text):
  for character in text:
    sys.stdout.write(character)
    sys.stdout.flush()
    time.sleep(0.05)

> PromptTemplates from LangChain for TinyDolphin

In [ ]:
# @title Prompt_Templates from LAngChain for TinyDolphin 1.1B
from langchain.prompts import PromptTemplate
# Templates for TinyDolphin
# tmplt_TDolphin_summary.format(context=yourcontext)
tmplt_TDolphin_summary = PromptTemplate.from_template(
    """<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Given this context:
[context]{context}
[end of context]

- SUMMARY:
<|im_end|>
<|im_start|>assistant"""
)
# tmplt_TDolphin_3topics.format(context=yourcontext)
tmplt_TDolphin_3topics = PromptTemplate.from_template(
    """<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Given this context:
[context]{context}
[end of context]

List the three main topics from the context:
<|im_end|>
<|im_start|>assistant"""
)
# tmplt_TDolphin_3questions.format(context=yourcontext)
tmplt_TDolphin_3questions = PromptTemplate.from_template(
    """<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Given this context:
[context]{context}
[end of context]

Generate three concise questions about the text.
<|im_end|>
<|im_start|>assistant"""
)
# tmplt_TDolphin_RAG.format(context=yourcontext, query=yourqeustion)
tmplt_TDolphin_RAG = PromptTemplate.from_template(
    """<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Given this context:
[context]{context}
[end of context]

Use the provided context to reply the user question:
{query}
<|im_end|>
<|im_start|>assistant"""
)

load document and chunck for FAQ preparartions

In [ ]:
# reading text file into string
with open('/content/Maybe Embeddings is what you need.txt', encoding='utf-8') as f:
  article = f.read()
f.close()
#tokens in the article
import tiktoken
encoding = tiktoken.get_encoding("r50k_base")
console.print(f"Tokens in the article: {len(encoding.encode(article))}")
#Preparing questions with big granularity
from langchain.text_splitter import TokenTextSplitter
TOKENtext_splitter = TokenTextSplitter(chunk_size=400, chunk_overlap=50)
q_context = TOKENtext_splitter.split_text(article) #create a list
console.print(f"Number of documents: {len(q_context)}")

Tokens in the article: 3028

Number of documents: 9

In [ ]:
console.print(f"Number of documents: {len(q_context)}")

Number of documents: 9

### Testing the tinest CrossReranker exsting - ms-marco-TinyBERT-L-2

In [6]:
from sentence_transformers import CrossEncoder

# Load the model, here we use our base sized model
model = CrossEncoder("cross-encoder/ms-marco-TinyBERT-L-2")  # 17 MegaByte !!

# Lets get the scores
query = 'Where is Windscribe located?'
results_q = model.rank(query, questions, return_documents=True, top_k=3)
results_a = model.rank(query, answers, return_documents=True, top_k=3)

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.6M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/543 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [7]:
def print_ranks(model_rank, title):
  #https://rich.readthedocs.io/en/stable/tables.html
  from rich.table import Table
  table = Table(title=title, show_lines=True)

  table.add_column("Score", justify="right", style="magenta", no_wrap=True)
  table.add_column("Text", justify="left", style="cyan")
  table.add_column("Doc_id", justify="center", style="cyan")
  #table.add_column("Box Office", justify="right", style="green")
  escore = f"{model_rank[0]['score']:.4f}"
  table.add_row(escore, model_rank[0]['text'],f"{model_rank[0]['corpus_id']}",style="red1 bold")
  for hit in model_rank[1:]:
    esscore = f"{hit['score']:.4f}"
    table.add_row(esscore, hit['text'], f"{hit['corpus_id']}")
  console.print(table)

In [8]:
print_ranks(results_a, "ms-marco-TinyBERT-L-2")
print_ranks(results_q, "ms-marco-TinyBERT-L-2")

                                            ms-marco-TinyBERT-L-2                                             
┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃  Score ┃ Text                                                                                     ┃ Doc_id ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0.9565 │ The actual Windscribe headquarters is based in Toronto, Ontario, Canada. However, we are │   0    │
│        │ a diverse company with employees working on three continents, in over 7 countries.       │        │
├────────┼──────────────────────────────────────────────────────────────────────────────────────────┼────────┤
│ 0.1326 │ In order to use Windscribe, you must have an existing internet connection, such as       │   6    │
│        │ mobile data, home internet or public WiFi. Windscribe does not provide an internet       │        │
│        │ connection, we simply encrypt and reroute your existing internet connection through our  │        │
│        │ secure VPN servers. This means that the bandwidth you use with the VPN is also being     │        │
│        │ used with your Internet Service Provider (ISP).                                          │        │
├────────┼──────────────────────────────────────────────────────────────────────────────────────────┼────────┤
│ 0.0483 │ Windscribe's encryption varies based on the protocol selected, as well as the format of  │   2    │
│        │ our app you are using: OpenVPN: Our OpenVPN implementation uses the AES-256-GCM cipher   │        │
│        │ with SHA512 auth and a 4096-bit RSA key. Perfect forward secrecy is also supported.      │        │
│        │ IKEv2: Our in-app IKEv2 implementation utilizes AES-256-GCM for encryption, SHA-256 for  │        │
│        │ integrity checks. Desktop and Android apps use ECP384 for Diffie-Hellman key negotiation │        │
│        │ (DH group 20), and iOS uses ECP521 for Diffie-Hellman key negotiation (DH group 21).     │        │
│        │ WireGuard: WireGuard is an opinionated protocol that uses ChaCha20 for symmetric         │        │
│        │ encryption, authenticated with Poly1305; Curve25519 for ECDH; BLAKE2s for hashing and    │        │
│        │ keyed hashing; SipHash24 for hashtable keys; and HKDF for key derivation.                │        │
└────────┴──────────────────────────────────────────────────────────────────────────────────────────┴────────┘

                          ms-marco-TinyBERT-L-2                           
┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃  Score ┃ Text                                                 ┃ Doc_id ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0.9834 │ Where is Windscribe located?                         │   0    │
├────────┼──────────────────────────────────────────────────────┼────────┤
│ 0.1294 │ What kind of encryption does Windscribe use?         │   2    │
├────────┼──────────────────────────────────────────────────────┼────────┤
│ 0.0226 │ Can I use Windscribe without an internet connection? │   6    │
└────────┴──────────────────────────────────────────────────────┴────────┘

In [9]:
console.print("Hello! How can I help you today?")
q = input("User: ")
results_a = model.rank(q, answers, return_documents=True, top_k=3)
id = results_a[0]['corpus_id']
typingPrint(answers[id])


Hello! How can I help you today?

User: where is Windscribe?
The actual Windscribe headquarters is based in Toronto, Ontario, Canada. However, we are a diverse company with employees working on three continents, in over 7 countries.

In [13]:
console.print("Hello! How can I help you today?")
q = input("User: ")
results_a = model.rank(q, answers, return_documents=True, top_k=3)
id = results_a[0]['corpus_id']
typingPrint(answers[id])

Hello! How can I help you today?

User: what are Windscribe's encryptions?
Windscribe's encryption varies based on the protocol selected, as well as the format of our app you are using: OpenVPN: Our OpenVPN implementation uses the AES-256-GCM cipher with SHA512 auth and a 4096-bit RSA key. Perfect forward secrecy is also supported. IKEv2: Our in-app IKEv2 implementation utilizes AES-256-GCM for encryption, SHA-256 for integrity checks. Desktop and Android apps use ECP384 for Diffie-Hellman key negotiation (DH group 20), and iOS uses ECP521 for Diffie-Hellman key negotiation (DH group 21). WireGuard: WireGuard is an opinionated protocol that uses ChaCha20 for symmetric encryption, authenticated with Poly1305; Curve25519 for ECDH; BLAKE2s for hashing and keyed hashing; SipHash24 for hashtable keys; and HKDF for key derivation.